In [2]:
%matplotlib inline

In [3]:

import os
import numpy as np
import json
from ipywidgets import interact
import matplotlib.pyplot as plt
import torch
import random
import re
from tqdm.auto import tqdm
from espnet.asr.pytorch_backend.asr import load_trained_model
if 'cd' not in globals():
    cd = True
    os.chdir('..')
import matplotlib
matplotlib.use('TkAgg')

In [8]:

i = 30
model, train_args = load_trained_model(f'exp/train_pytorch_wav2vecfex/results/snapshot.ep.{i}')
device = torch.device('cuda')
model = model.float()
model = model.to(device)



In [53]:
len(uniq)

15

In [54]:
with open('dump/train/deltafalse/data.json.npy') as f:
    js = json.load(f)['utts']

In [9]:
def get_lang(d):
    s = d.split('_')[0]
    s = re.sub(r'\d+$', '', s.split('-')[0]) if re.search('[a-zA-Z]+', s) else s
    return s

In [13]:
n_sample = 1000
frame_ratio = 0.1

In [75]:
x.size()

torch.Size([35839])

In [77]:
torch.as_tensor(x)

tensor([[-2.4414e-04, -1.8311e-04,  2.4414e-04,  ...,  1.8311e-04,
         -6.1035e-05, -1.5259e-04]], device='cuda:0')

In [79]:
xs = []
ys = []
random.seed(1)
with torch.no_grad():
    for k, v in tqdm(random.sample(js.items(), n_sample)):
        lang = get_lang(k)
#         x = torch.FloatTensor(np.load(v['input'][0]['feat'])).unsqueeze(0).to(device)
        x = torch.FloatTensor(np.load(v['input'][0]['feat'])).to(device)
        features = model.encode(x)
        
        features = features.squeeze(0).detach().cpu().numpy()
        T, d = features.shape
        n_frame = int(frame_ratio * T)
        ys.extend([lang] * n_frame)
        idx = random.sample(list(range(T)), n_frame)
        xs.append(features[idx])
#         break
    xs = np.vstack(xs)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [27]:
def to_onehot(ys):
    uniq = sorted(list(np.unique(ys)))
    
    out = np.zeros((len(ys), len(uniq)))
    l2int = {l: i for i, l in enumerate(uniq)}
    print(l2int)
    for i, l in enumerate(ys):
        out[i, l2int[l]] = 1
    return out

In [101]:
uniq = sorted(list(np.unique(ys)))
uniq

['103',
 '107',
 '206',
 '307',
 '402',
 '404',
 'BG',
 'CH',
 'CZ',
 'FR',
 'GE',
 'N',
 'PO',
 'TH',
 'TU']

In [62]:
def to_int(ys):
    uniq = sorted(list(np.unique(ys)))
    
#     out = np.zeros((len(ys), len(uniq)))
    l2int = {l: i for i, l in enumerate(uniq)}
    print(l2int)
    out = np.array([l2int[l] for i, l in enumerate(ys)])
    return out

In [81]:
X = np.array(xs)
Y = to_int(ys)
X.shape, Y.shape

{'103': 0, '107': 1, '206': 2, '307': 3, '402': 4, '404': 5, 'BG': 6, 'CH': 7, 'CZ': 8, 'FR': 9, 'GE': 10, 'N': 11, 'PO': 12, 'TH': 13, 'TU': 14}


((22658, 256), (22658,))

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [85]:
clf = LinearSVC(random_state=0, max_iter=5000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

/home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [90]:

# encode
clf = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.9880847308031774


/home/hertin/Software/miniconda3/envs/espnet-lowres/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [106]:
from pprint import pprint
uniq = sorted(list(np.unique(ys)))

acc = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average=None)

print('acc', acc, 'f1_macro', f1_macro, 'f1_micro', f1_micro)
pprint(list(zip(uniq, f1)))

acc 0.9880847308031774 f1_macro 0.9910002299893931 f1_micro 0.9880847308031774
[('103', 0.9811827956989247),
 ('107', 0.9846827133479211),
 ('206', 0.9875),
 ('307', 0.9857954545454546),
 ('402', 0.9679144385026738),
 ('404', 0.9837133550488599),
 ('BG', 1.0),
 ('CH', 0.9931972789115646),
 ('CZ', 1.0),
 ('FR', 0.9957805907172996),
 ('GE', 1.0),
 ('N', 0.9986320109439125),
 ('PO', 0.9936305732484076),
 ('TH', 0.9929742388758782),
 ('TU', 1.0)]


In [ ]:
# feature extractor
acc = accuracy_score(y_test, y_pred)

base 0.9880847308031774 lgcn 0.9995374653098983 lemb 0.9993061979648473


In [ ]:
lgcn+lagn 0.5062442183163737 lemb+lagn 0.7414431082331174

In [ ]:
# lgcn 0
acc 0.5062442183163737 f1_macro 0.462408857271572 f1_micro 0.5062442183163737
[('103', 0.4207920792079208),
 ('107', 0.6154847104749512),
 ('206', 0.46926229508196726),
 ('307', 0.3880597014925374),
 ('402', 0.32902033271719033),
 ('404', 0.5421558164354322),
 ('BG', 0.3891891891891892),
 ('CH', 0.5130434782608695),
 ('CZ', 0.3595505617977528),
 ('FR', 0.457286432160804),
 ('GE', 0.32335329341317365),
 ('N', 0.6835187057633972),
 ('PO', 0.32558139534883723),
 ('TH', 0.6108374384236454),
 ('TU', 0.5089974293059126)]

In [ ]:
lgcn N
acc 0.5238205365402405 f1_macro 0.4922570593656955 f1_micro 0.5238205365402405
[('103', 0.38944723618090454),
 ('107', 0.5768976897689769),
 ('206', 0.44691607684529827),
 ('307', 0.3955696202531645),
 ('402', 0.3404255319148936),
 ('404', 0.5520361990950224),
 ('BG', 0.44198895027624313),
 ('CH', 0.496),
 ('CZ', 0.40714285714285714),
 ('FR', 0.5626373626373626),
 ('GE', 0.4023668639053254),
 ('N', 0.8602620087336244),
 ('PO', 0.3157894736842105),
 ('TH', 0.6278481012658228),
 ('TU', 0.5685279187817259)]



In [ ]:
                         | lemb
audio -> featureextractor -> encoder -> ctc
                         |-> 46 %    |
                                     |-> 98 / 99
                                     |-> 30 - 90

In [ ]:
# lemb 0
[('103', 0.5192307692307692),
 ('107', 0.8364153627311522),
 ('206', 0.6685823754789272),
 ('307', 0.6390685640362225),
 ('402', 0.5368248772504092),
 ('404', 0.7046263345195729),
 ('BG', 0.9463414634146341),
 ('CH', 0.9008264462809917),
 ('CZ', 0.8553459119496856),
 ('FR', 0.7598039215686274),
 ('GE', 0.736842105263158),
 ('N', 0.9328782707622297),
 ('PO', 0.5439330543933055),
 ('TH', 0.8956043956043955),
 ('TU', 0.8443271767810027)]

penalize the language clf loss
language model training data
should phoneme boundary change depend on a/i/ei, 
replacing the phoible phoneme with langnet phoneme


In [ ]:

lemb N
acc 0.6667437557816837 f1_macro 0.6393575378750671 f1_micro 0.6667437557816837
[('103', 0.5204359673024523),
 ('107', 0.7983134223471539),
 ('206', 0.5985985985985987),
 ('307', 0.5620542082738944),
 ('402', 0.504950495049505),
 ('404', 0.6666666666666667),
 ('BG', 0.8095238095238095),
 ('CH', 0.6419753086419754),
 ('CZ', 0.5752508361204013),
 ('FR', 0.6057692307692307),
 ('GE', 0.5227272727272727),
 ('N', 0.9234184239733628),
 ('PO', 0.4232365145228215),
 ('TH', 0.7537688442211056),
 ('TU', 0.6836734693877552)]


In [11]:
'''
lemblinear: w2v + linear
acc 0.45050878815911194 f1_macro 0.4387434699442588 f1_micro 0.450508788159112
'''
f1_dict = OrderedDict([('103', 0.2857142857142857),
 ('107', 0.4832451499118166),
 ('206', 0.342451874366768),
 ('307', 0.2576489533011272),
 ('402', 0.167420814479638),
 ('404', 0.4682203389830508),
 ('BG', 0.7419354838709677),
 ('CH', 0.2119205298013245),
 ('CZ', 0.4892086330935252),
 ('FR', 0.3937007874015748),
 ('GE', 0.5391304347826087),
 ('N', 0.6977973568281938),
 ('PO', 0.31527093596059114),
 ('TH', 0.7109974424552431),
 ('TU', 0.47648902821316613)])
np.std(list(f1_dict.values())), np.mean(list(f1_dict.values()))

(0.1747148139246091, 0.4387434699442588)

In [ ]:
acc 0.7717391304347826 f1_macro 0.7520866120652767 f1_micro 0.7717391304347826
[('103', 0.726161369193154),
 ('107', 0.864748201438849),
 ('206', 0.7109375000000001),
 ('307', 0.7085714285714286),
 ('402', 0.5017182130584192),
 ('404', 0.8312849162011173),
 ('BG', 0.71875),
 ('CH', 0.7983193277310925),
 ('CZ', 0.6529209621993127),
 ('FR', 0.7476635514018691),
 ('GE', 0.7177033492822967),
 ('N', 0.9622857142857143),
 ('PO', 0.7383512544802867),
 ('TH', 0.8268156424581005),
 ('TU', 0.7750677506775068)]

In [5]:
import numpy as np
from collections import OrderedDict


In [12]:
'''
lemblinear: w2v only
acc 0.45050878815911194 f1_macro 0.4340926304476996 f1_micro 0.450508788159112
'''
f1_dict = OrderedDict([('103', 0.3057503506311361),
 ('107', 0.4759106933019976),
 ('206', 0.3522267206477733),
 ('307', 0.2709030100334448),
 ('402', 0.1748878923766816),
 ('404', 0.46170212765957447),
 ('BG', 0.6892655367231638),
 ('CH', 0.1276595744680851),
 ('CZ', 0.5058365758754865),
 ('FR', 0.38764044943820225),
 ('GE', 0.5833333333333334),
 ('N', 0.690295358649789),
 ('PO', 0.3380281690140845),
 ('TH', 0.6913580246913581),
 ('TU', 0.45659163987138257)])
np.std(list(f1_dict.values())), np.mean(list(f1_dict.values()))

(0.17313864857382286, 0.4340926304476996)

In [13]:

'''
Non-parametric computation of entropy and mutual-information
Adapted by G Varoquaux for code created by R Brette, itself
from several papers (see in the code).
This code is maintained at https://github.com/mutualinfo/mutual_info 
Please download the latest code there, to have improvements and 
bug fixes.
These computations rely on nearest-neighbor statistics
'''
import numpy as np

from scipy.special import gamma,psi
from scipy import ndimage
from scipy.linalg import det
from numpy import pi

from sklearn.neighbors import NearestNeighbors

__all__=['entropy', 'mutual_information', 'entropy_gaussian']

EPS = np.finfo(float).eps


def nearest_distances(X, k=1):
    '''
    X = array(N,M)
    N = number of points
    M = number of dimensions
    returns the distance to the kth nearest neighbor for every point in X
    '''
    knn = NearestNeighbors(n_neighbors=k + 1)
    knn.fit(X)
    d, _ = knn.kneighbors(X) # the first nearest neighbor is itself
    return d[:, -1] # returns the distance to the kth nearest neighbor


def entropy_gaussian(C):
    '''
    Entropy of a gaussian variable with covariance matrix C
    '''
    if np.isscalar(C): # C is the variance
        return .5*(1 + np.log(2*pi)) + .5*np.log(C)
    else:
        n = C.shape[0] # dimension
        return .5*n*(1 + np.log(2*pi)) + .5*np.log(abs(det(C)))


def entropy(X, k=1):
    ''' Returns the entropy of the X.
    Parameters
    ===========
    X : array-like, shape (n_samples, n_features)
        The data the entropy of which is computed
    k : int, optional
        number of nearest neighbors for density estimation
    Notes
    ======
    Kozachenko, L. F. & Leonenko, N. N. 1987 Sample estimate of entropy
    of a random vector. Probl. Inf. Transm. 23, 95-101.
    See also: Evans, D. 2008 A computationally efficient estimator for
    mutual information, Proc. R. Soc. A 464 (2093), 1203-1215.
    and:
    Kraskov A, Stogbauer H, Grassberger P. (2004). Estimating mutual
    information. Phys Rev E 69(6 Pt 2):066138.
    '''

    # Distance to kth nearest neighbor
    r = nearest_distances(X, k) # squared distances
    n, d = X.shape
    volume_unit_ball = (pi**(.5*d)) / gamma(.5*d + 1)
    '''
    F. Perez-Cruz, (2008). Estimation of Information Theoretic Measures
    for Continuous Random Variables. Advances in Neural Information
    Processing Systems 21 (NIPS). Vancouver (Canada), December.
    return d*mean(log(r))+log(volume_unit_ball)+log(n-1)-log(k)
    '''
    return (d*np.mean(np.log(r + np.finfo(X.dtype).eps))
            + np.log(volume_unit_ball) + psi(n) - psi(k))


def mutual_information(variables, k=1):
    '''
    Returns the mutual information between any number of variables.
    Each variable is a matrix X = array(n_samples, n_features)
    where
      n = number of samples
      dx,dy = number of dimensions
    Optionally, the following keyword argument can be specified:
      k = number of nearest neighbors for density estimation
    Example: mutual_information((X, Y)), mutual_information((X, Y, Z), k=5)
    '''
    if len(variables) < 2:
        raise AttributeError(
                "Mutual information must involve at least 2 variables")
    all_vars = np.hstack(variables)
    return (sum([entropy(X, k=k) for X in variables])
            - entropy(all_vars, k=k))


def mutual_information_2d(x, y, sigma=1, normalized=False):
    """
    Computes (normalized) mutual information between two 1D variate from a
    joint histogram.
    Parameters
    ----------
    x : 1D array
        first variable
    y : 1D array
        second variable
    sigma: float
        sigma for Gaussian smoothing of the joint histogram
    Returns
    -------
    nmi: float
        the computed similariy measure
    """
    bins = (256, 256)

    jh = np.histogram2d(x, y, bins=bins)[0]

    # smooth the jh with a gaussian filter of given sigma
    ndimage.gaussian_filter(jh, sigma=sigma, mode='constant',
                                 output=jh)

    # compute marginal histograms
    jh = jh + EPS
    sh = np.sum(jh)
    jh = jh / sh
    s1 = np.sum(jh, axis=0).reshape((-1, jh.shape[0]))
    s2 = np.sum(jh, axis=1).reshape((jh.shape[1], -1))

    # Normalised Mutual Information of:
    # Studholme,  jhill & jhawkes (1998).
    # "A normalized entropy measure of 3-D medical image alignment".
    # in Proc. Medical Imaging 1998, vol. 3338, San Diego, CA, pp. 132-143.
    if normalized:
        mi = ((np.sum(s1 * np.log(s1)) + np.sum(s2 * np.log(s2)))
                / np.sum(jh * np.log(jh))) - 1
    else:
        mi = ( np.sum(jh * np.log(jh)) - np.sum(s1 * np.log(s1))
               - np.sum(s2 * np.log(s2)))

    return mi




In [2]:

import os
import numpy as np
import json
from ipywidgets import interact
import matplotlib.pyplot as plt
import torch
import random
import re
from tqdm.auto import tqdm
from espnet.asr.pytorch_backend.asr import load_trained_model
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.neighbors.kde import KernelDensity
if 'cd' not in globals():
    cd = True
    os.chdir('..')
import matplotlib

In [3]:


i = 30
model, train_args = load_trained_model(f'exp/train_pytorch_wav2vecfexlemblinear/results/snapshot.ep.{i}')
device = torch.device('cuda:1')
model = model.float()
model = model.to(device)

with open('dump/train/deltafalse/data.json.npy') as f:
    js = json.load(f)['utts']
    
def get_lang(d):
    s = d.split('_')[0]
    s = re.sub(r'\d+$', '', s.split('-')[0]) if re.search('[a-zA-Z]+', s) else s
    return s

def to_onehot(ys):
    uniq = sorted(list(np.unique(ys)))
    
    out = np.zeros((len(ys), len(uniq)))
    l2int = {l: i for i, l in enumerate(uniq)}
    print(l2int)
    for i, l in enumerate(ys):
        out[i, l2int[l]] = 1
    return out

def to_int(ys):
    uniq = sorted(list(np.unique(ys)))
    
#     out = np.zeros((len(ys), len(uniq)))
    l2int = {l: i for i, l in enumerate(uniq)}
    print(l2int)
    out = np.array([l2int[l] for i, l in enumerate(ys)])
    return out
n_sample = 1000
frame_ratio = 0.1

In [10]:

xs = []
ys = []
random.seed(2)
with torch.no_grad():
    for k, v in tqdm(random.sample(js.items(), n_sample)):
        lang = get_lang(k)
        x = torch.FloatTensor(np.load(v['input'][0]['feat'])).to(device)
        # features = model.encode(x, lang_labels=['N'])
        features = model.feature_extractor(x.unsqueeze(0))
        features = features.transpose(1, 2)
        features = model.feature_mapping(features)
        
        features = features.squeeze(0).detach().cpu().numpy()
        T, d = features.shape
        n_frame = int(frame_ratio * T)
        ys.extend([lang] * n_frame)
        idx = random.sample(list(range(T)), n_frame)
        xs.append(features[idx])
#         break
    xs = np.vstack(xs)
X = np.array(xs)
Y = to_int(ys) 

  0%|          | 0/1000 [00:00<?, ?it/s]

{'103': 0, '107': 1, '206': 2, '307': 3, '402': 4, '404': 5, 'BG': 6, 'CH': 7, 'CZ': 8, 'FR': 9, 'GE': 10, 'N': 11, 'PO': 12, 'TH': 13, 'TU': 14}


In [11]:
np.random.seed(2)
idx = np.random.choice(list(range(len(X))), size=1000, replace=False)
X = X[idx]
X.shape

(1000, 512)

In [12]:
### create data ###
sample_count, n = X.shape
data = X
data_norm = np.sqrt(np.sum(data*data, axis=1))
data = data/data_norm[:, None]   # Normalized data to be on unit sphere

data.shape

(1000, 512)

In [23]:
## estimate pdf using KDE with gaussian kernel
# kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(data)

# log_p = kde.score_samples(data)  # returns log(p) of data sample
# p = np.exp(log_p)                # estimate p of data sample
# entropy = -np.sum(p*log_p)       # evaluate entropy
# entropy

-1.198765336660309e+156

In [13]:
# point wise distance lembliner: w2v + linear
np.sum(euclidean_distances(X, X))

1558339.6

In [14]:
# point wise distance lembliner: w2v + linear
np.sum(euclidean_distances(data, data))

1196159.5

In [8]:
# point wise distance lembliner: w2v
np.sum(euclidean_distances(X, X))

447365.0

In [9]:
# point wise distance lembliner: w2v
np.sum(euclidean_distances(data, data))

1369082.2

In [18]:
def get_samples(path, linear=False):
    model, train_args = load_trained_model(path)
    device = torch.device('cuda:1')
    model = model.float()
    model = model.to(device)
    
    xs = []
    ys = []
    random.seed(2)
    with torch.no_grad():
        for k, v in tqdm(random.sample(js.items(), n_sample)):
            lang = get_lang(k)
            x = torch.FloatTensor(np.load(v['input'][0]['feat'])).to(device)
            # features = model.encode(x, lang_labels=['N'])
            features = model.feature_extractor(x.unsqueeze(0))
            features = features.transpose(1, 2)
            if linear:
                features = model.feature_mapping(features)

            features = features.squeeze(0).detach().cpu().numpy()
            T, d = features.shape
            n_frame = int(frame_ratio * T)
            ys.extend([lang] * n_frame)
            idx = random.sample(list(range(T)), n_frame)
            xs.append(features[idx])
    #         break
        xs = np.vstack(xs)
    X = np.array(xs)
    Y = to_int(ys)
    return X, Y

In [19]:
i = 30
path = f'exp/train_pytorch_wav2vecfexlemblinear/results/snapshot.ep.{i}'
X, Y = get_samples(path, linear=False)
np.random.seed(2)
idx = np.random.choice(list(range(len(X))), size=1000, replace=False)
X = X[idx]

sample_count, n = X.shape
data = X
data_norm = np.sqrt(np.sum(data*data, axis=1))
data = data/data_norm[:, None]   # Normalized data to be on unit sphere

print(np.sum(euclidean_distances(X, X)), np.sum(euclidean_distances(data, data)))

  0%|          | 0/1000 [00:00<?, ?it/s]

{'103': 0, '107': 1, '206': 2, '307': 3, '402': 4, '404': 5, 'BG': 6, 'CH': 7, 'CZ': 8, 'FR': 9, 'GE': 10, 'N': 11, 'PO': 12, 'TH': 13, 'TU': 14}
447365.0 1369082.2


In [20]:
i = 30
path = f'exp/train_pytorch_wav2vecfexlemblinear/results/snapshot.ep.{i}'
X, Y = get_samples(path, linear=True)
np.random.seed(2)
idx = np.random.choice(list(range(len(X))), size=1000, replace=False)
X = X[idx]

sample_count, n = X.shape
data = X
data_norm = np.sqrt(np.sum(data*data, axis=1))
data = data/data_norm[:, None]   # Normalized data to be on unit sphere

print(np.sum(euclidean_distances(X, X)), np.sum(euclidean_distances(data, data)))

  0%|          | 0/1000 [00:00<?, ?it/s]

{'103': 0, '107': 1, '206': 2, '307': 3, '402': 4, '404': 5, 'BG': 6, 'CH': 7, 'CZ': 8, 'FR': 9, 'GE': 10, 'N': 11, 'PO': 12, 'TH': 13, 'TU': 14}
1558339.6 1196159.5


In [ ]:
w2v
447365.0 normalized 1369082.2

In [ ]:
w2v + linear
1558339.6 normalized 1196159.5 entropy - 

In [ ]:
w2v + adverserial linear
1795634.6 normalized 1375990.0 entropy +

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score


In [50]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

clf = LinearSVC(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

from pprint import pprint
uniq = sorted(list(np.unique(ys)))

acc = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average=None)

print('acc', acc, 'f1_macro', f1_macro, 'f1_micro', f1_micro)
pprint(list(zip(uniq, f1)))

acc 0.45050878815911194 f1_macro 0.4340926304476996 f1_micro 0.450508788159112
[('103', 0.3057503506311361),
 ('107', 0.4759106933019976),
 ('206', 0.3522267206477733),
 ('307', 0.2709030100334448),
 ('402', 0.1748878923766816),
 ('404', 0.46170212765957447),
 ('BG', 0.6892655367231638),
 ('CH', 0.1276595744680851),
 ('CZ', 0.5058365758754865),
 ('FR', 0.38764044943820225),
 ('GE', 0.5833333333333334),
 ('N', 0.690295358649789),
 ('PO', 0.3380281690140845),
 ('TH', 0.6913580246913581),
 ('TU', 0.45659163987138257)]


In [48]:
# accs lemblinear: w2v + linear
accs = []
for l in uniq:
    i = l2int[l]
    acc = np.sum((y_pred==i) & (y_test==i)) / np.sum(y_test==i)
    accs.append(acc)
np.var(accs), np.std(accs), np.mean(accs)

(0.05399172289086165, 0.23236119058668478, 0.4375800906788145)

In [51]:
# accs lemblinear: w2v
accs = []
for l in uniq:
    i = l2int[l]
    acc = np.sum((y_pred==i) & (y_test==i)) / np.sum(y_test==i)
    accs.append(acc)
np.var(accs), np.std(accs), np.mean(accs)

(0.05512916208483968, 0.23479600099839792, 0.4305877113086551)

In [52]:
y_test

array([13,  8,  1, ..., 10,  1,  1])

In [41]:
l2int = {l: i for i, l in enumerate(uniq)}

{'103': 0,
 '107': 1,
 '206': 2,
 '307': 3,
 '402': 4,
 '404': 5,
 'BG': 6,
 'CH': 7,
 'CZ': 8,
 'FR': 9,
 'GE': 10,
 'N': 11,
 'PO': 12,
 'TH': 13,
 'TU': 14}